***Эксперимент 3***

*Сравнение изображения лица анфас с изображениями с разных ракурсов Цель: оценить, как разные модели справляются с изображениями повёрнутой головы Гипотеза: если дистанции между изображением лица анфас и изображениями повернутой головы малы, значит модель работает хорошо.*

In [ ]:
!pip install deepface mediapipe
!pip install matplotlib

In [3]:
import cv2
import time
import os
import glob
from PIL import Image, ImageOps, ImageDraw
from deepface import DeepFace
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import ipyplot

2024-04-18 11:43:37.850834: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-18 11:43:37.850894: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-18 11:43:37.852777: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-18 11:43:37.970767: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
input_folder = '/tf/data/testt'

In [5]:
model_names = [
    "VGG-Face",
    "Facenet",
    "Facenet512",
    "OpenFace",
    "DeepFace",
    "DeepID",
    "ArcFace",
    "SFace"
]
thresholds = {
     "VGG-Face": 1.17,
    "Facenet": 0.8,
    "Facenet512": 1.04,
    "OpenFace": 0.55,
    "DeepFace": 0.64,
    "DeepID": 0.17,
    "ArcFace": 1.13,
    "SFace": 1.055,
}
model_resolutions = {
    "VGG-Face": (224, 224),
    "Facenet": (220, 220),
    "Facenet512": (160, 160),
    "OpenFace": (96, 96),
    "DeepFace": (152, 152),
    "DeepID": (55, 47),
    "ArcFace": (112, 112),
    "SFace": (112, 112)
}
detector_backend = "retinaface"

In [1]:
def crop_to_square_pil(image, face_info):

    x, y, w, h = face_info['facial_area']['x'], face_info['facial_area']['y'], face_info['facial_area']['w'], face_info['facial_area']['h']

    square_size = max(w, h)
    new_x = max(0, x)
    new_y = max(0, y)
    cropped_image = image.crop((new_x - (h-w)//2, new_y, new_x - (h-w)//2 + square_size, new_y + square_size))

    return cropped_image

def resize_image_pil(image, new_resolution):
    resized_img = image.resize(new_resolution, Image.LANCZOS)
    return resized_img
    
def save_to_csv(data, columns, filename):
    df = pd.DataFrame(data, columns=columns)
    df.to_csv(filename + '.csv', index=False)

In [ ]:
users_list = os.listdir(os.path.join(input_folder, f"center/center/"))
horizontal_positions = ["45_left","center","45_right"]
vertical_positions = ["up","center","down"]

for model_name, resolution in model_resolutions.items():
    print(f"model: {model_name}")
    df_results_model = []

    for person_name in tqdm(users_list):
        person_distances = {"up": {"45_left": 0, "center": 0, "45_right": 0},
                            "center": {"45_left": 0, "center": 0, "45_right": 0},
                            "down": {"45_left": 0, "center": 0, "45_right": 0}}

        image_anfas_no_glasses_path = os.path.join(input_folder, f"center/center/{person_name}/no_glasses.JPG")
        if not os.path.exists(image_anfas_no_glasses_path):
            continue
    
        image_anfas_no_glasses = Image.open(image_anfas_no_glasses_path)
        image_anfas_no_glasses = ImageOps.exif_transpose(image_anfas_no_glasses)
        face_info_anfas_no_glasses = DeepFace.extract_faces(img_path=image_anfas_no_glasses_path, detector_backend=detector_backend)
        cropped_image_anfas_no_glasses = crop_to_square_pil(image_anfas_no_glasses, face_info_anfas_no_glasses[0])
        resized_image_anfas_no_glasses = resize_image_pil(cropped_image_anfas_no_glasses, resolution)
        image_anfas_no_glasses = np.array(resized_image_anfas_no_glasses)

        image_anfas_glasses_path = os.path.join(input_folder, f"center/center/{person_name}/glasses.JPG")
        yesGlasses = os.path.exists(image_anfas_glasses_path)

        for vertical_position in vertical_positions:
            for horizontal_position in horizontal_positions:
                if vertical_position == horizontal_position:
                    continue
                image_current_no_glasses_path = os.path.join(input_folder, f"{vertical_position}/{horizontal_position}/{person_name}/no_glasses.JPG")
                if not os.path.exists(image_current_no_glasses_path):
                    continue
                image_current_no_glasses = Image.open(image_current_no_glasses_path)
                image_current_no_glasses = ImageOps.exif_transpose(image_current_no_glasses)
                face_info_current_no_glasses = DeepFace.extract_faces(img_path=image_current_no_glasses_path, detector_backend=detector_backend)
                cropped_image_current_no_glasses = crop_to_square_pil(image_current_no_glasses, face_info_current_no_glasses[0])
                resized_image_current_no_glasses = resize_image_pil(cropped_image_current_no_glasses, resolution)
                image_current_no_glasses = np.array(resized_image_current_no_glasses)
                # display(resized_image_current_no_glasses)

                result = DeepFace.verify(
                    img1_path=image_anfas_no_glasses,
                    img2_path=image_current_no_glasses,
                    enforce_detection=False,
                    detector_backend=detector_backend,
                    model_name=model_name,
                    distance_metric="euclidean_l2"
                )
                # print(f"real run: comparing {vertical_position}/{horizontal_position}/{person_name}/no_glasses.JPG with center/center/{person_name}/no_glasses.JPG, distance is {result['distance']}")
                person_distances[vertical_position][horizontal_position] = result['distance']

                if yesGlasses:
                    image_anfas_glasses = Image.open(image_anfas_glasses_path)
                    image_anfas_glasses = ImageOps.exif_transpose(image_anfas_glasses)
                    face_info_anfas_glasses = DeepFace.extract_faces(img_path=image_anfas_glasses_path, detector_backend=detector_backend)
                    cropped_image_anfas_glasses = crop_to_square_pil(image_anfas_glasses, face_info_anfas_glasses[0])
                    resized_image_anfas_glasses = resize_image_pil(cropped_image_anfas_glasses, resolution)
                    image_anfas_glasses = np.array(resized_image_anfas_glasses)
                   
                    image_current_glasses_path = os.path.join(input_folder, f"{vertical_position}/{horizontal_position}/{person_name}/glasses.JPG")
                    if not os.path.exists(image_current_glasses_path):
                        continue 
                    image_current_glasses = Image.open(image_current_glasses_path)
                    image_current_glasses = ImageOps.exif_transpose(image_current_glasses)
                    face_info_current_glasses = DeepFace.extract_faces(img_path=image_current_glasses_path, detector_backend=detector_backend)
                    cropped_image_current_glasses = crop_to_square_pil(image_current_glasses, face_info_current_glasses[0])
                    resized_image_current_glasses = resize_image_pil(cropped_image_current_glasses, resolution)
                    image_current_glasses = np.array(resized_image_current_glasses)
                    # display(resized_image_current_glasses)
                   
                    result_glasses = DeepFace.verify(
                        img1_path=image_anfas_glasses,
                        img2_path=image_current_glasses,
                        enforce_detection=False,
                        detector_backend=detector_backend,
                        model_name=model_name,
                        distance_metric="euclidean_l2"
                    )
                    # print(f"real run: comparing {vertical_position}/{horizontal_position}/{person_name}/glasses.JPG with center/center/{person_name}/glasses.JPG, distance is {result_glasses['distance']}")
                    person_distances[vertical_position][horizontal_position] = result_glasses['distance']
                    
                    df_results_model.append({
                        'model_name': model_name,
                        'person_name': person_name,
                        'vertical_position': vertical_position,
                        'horizontal_position': horizontal_position,
                        'distance': result_glasses['distance'],
                        'verified': result_glasses['verified'],
                        'glasses': 'yes'
                    })

                df_results_model.append({
                    'model_name': model_name,
                    'person_name': person_name,
                    'vertical_position': vertical_position,
                    'horizontal_position': horizontal_position,
                    'distance': result['distance'],
                    'verified': result['verified'],
                    'glasses': 'no'
                })
    
    df_results_model = pd.DataFrame(df_results_model)
    df_results_model.to_csv(f"{model_name}_results.csv", index=False)